# Advent of Code Day 14

Day 14 partially simulates a disk defragmenter.  In part one, we use the Knot Hash from Day 10 to produce a 128x128 grid.  To do so, we were given some input (as a string) and had to knot hash that value along with a row number.  After genering the knot hash, it was converted into a a binary representation (128 bits per row) where each 1 bit is a used square in the disk and each 0 bit is free.  Part one was concluded after figuring out how many used squares were in the disk.  For part two, we had to sweep the disk idenitfying contiguous regions of used squares and determine how many distinct regions were represented on the disk

In [ ]:
from knothash import KnotHash # the KnotHash class from Day 10 I moved to its own module

### Part One Helper Functions

To assist with part 1, it was necessary to define a few extra functions around KnotHash.  One, since KnotHash produces a hexadecimal version of the hash, I wrote a function to take that hexadecimal value and convert it to a binary string.  This is simple enough in Python, but I thought there was value in putting it behind a function so I could give it a meaningful name.  Also hex2bin takes care of padding the string (which it needs to be out to 128 to satisfy the problem ultimately). 

To keep KnotHash and Disk decoupled, I wrote a method to generate the binary string hashes for all the input rules using KnotHash.  The caller (can then create the Disk from that input).

In [ ]:
def hex2bin(hex_value, pad = 0):
    binary = '{0:b}'.format(int(hex_value, 16))
    
    return binary.zfill(pad)    



def compute_rowwise_binary_hashes(key_string_format):
    """ Given a string format that contains the key string plus a single format placeholder for substituting the iteration,
        returns an array of binary strings.  That is, each binary string is a binary number that corresponds to the hash
        computed from a distinct input.  Each binary string is intended to represent a row within a compute disk grid. 
        It will be in the form ['101001001011', '010101001100',....]
    """
    binary_hashes = []
    for row in range(0, 128):
        key_string = key_string_format.format(row)
        lengths = [ord(i) for i in key_string] + [17, 31, 73, 47, 23]

        knot_hash = KnotHash(range(0, 256), lengths)

        hashed = knot_hash.hex_hash(64, 16)

        binary_hashes.append(hex2bin(hashed, pad = 128))
        
    return binary_hashes

### Disk

Disk represents the 128x128 grid of used or free squares, primarily for part two.  It takes a collectoin of binary strings as input and stores those as a grid.  You can then use mark_regions to sweep the grid and put them into regions.  This version is crude and does not keep track of which squares are in which regions, it only keeps track of the number of regions.  Tracking the specific squares per region would be very easy though: just need to alter mark_regions to use a new dictionary to map a unique region number to the region squares.  

This was my first time experimenting with a few Python concepts:  __getitem__ and __setitem__.  I started with a function called at that took a coordinate tuple, but figured out the Pythonic way is with the get and set item dunders so I changed to using those.  My getitem is simplified and only allows a tuple and no slices, which is sufficient for this use case. 

In [ ]:
class Disk(object):
      
    def __init__(self, binary_string_grid):
        self.grid =[]
    
        for binary_hash in binary_string_grid:
            self.grid.append(['#' if b=='1' else '.' for b in binary_hash])
        
        self._top_left = (0, 0)
        self._bottom_right = (len(self.grid)-1, len(self.grid[0])-1)
        
        self._grid_size = self._bottom_right[0] - self._top_left[0] + 1
        self.region_count = 0
    
    def display(self):
           
        for row in self.grid:
            print ''.join(row)    
  
    def __getitem__(self, key):
        if isinstance(key, tuple):           
            return self.grid[key[0]][key[1]]
        else:           
            raise ValueError('__getitem__requires a (row, col) tuple')
    
    def __setitem__(self, key, value):   
            self.grid[key[0]][key[1]] = value
    
    def mark_regions(self):
       
        for row_index in range(0, self._grid_size):
            for col_index in range(0, self._grid_size):
                region_squares = self._mark_region((row_index, col_index), '*')
                
                if region_squares:
                    self.region_count += 1
      

    def _mark_region(self, start_square, region_symbol):
        if self[start_square] != '#':
            return []
        
        region_squares = self._find_adjacent_unmarked_squares(start_square)
       
        for square in region_squares:
            self[square] = region_symbol
            
        return region_squares
       
    def _find_adjacent_unmarked_squares(self, square):
        
        if self[square] != '#':
            return set([])

        squares_to_mark = set([square])

        adjacent_squares = self._get_adjacent_squares(square)
              
        while adjacent_squares:
            possible_square = adjacent_squares.pop()
           
            if self[possible_square] == '#':
                squares_to_mark.add(possible_square)
               
                adjacent_to_possible = self._get_adjacent_squares(possible_square)
                
                adjacent_squares = adjacent_squares.union(adjacent_to_possible).difference(squares_to_mark)

        return squares_to_mark
        
    
    def _get_adjacent_squares(self, square):       
    
        adjacent = []

        adjacent.append((square[0] - 1, square[1]))
        adjacent.append((square[0] + 1, square[1]))
        adjacent.append((square[0], square[1] - 1))
        adjacent.append((square[0], square[1] + 1))
        
        return set(self._clip(adjacent))
    
    def _clip(self, squares):        
        return [(x, y) for (x,y) in squares if (x >= self._top_left[0] and x <= self._bottom_right[0]) \
                                                 and (y >= self._top_left[1] and y <= self._bottom_right[1]) ]
    

In [ ]:
def solve_part_one():
    
    rows = compute_rowwise_binary_hashes('jzgqcdpd-{}')
    
    print as_grid(rows)
    
    used = sum([1 if s =='1' else 0 for row in rows for s in str(row) ])   
        
    print 'Used squares = {}'.format(used)
    
def solve_part_two():
    rows = compute_rowwise_binary_hashes('jzgqcdpd-{}')
    
    disk = Disk(rows)
    
    disk.mark_regions()
    
    print 'Region Count = {}'.format(disk.region_count)

In [ ]:
solve_part_two()